# Weaviate <> Google Gemini Ultra Integration Notebook

Welcome to the Weaviate and Google Gemini Ultra notebook. This notebook gets you up and running with the Weaviate and Gemini Ultra integration. 

![image](cover_photo1.png)


## Set up

### Jupyter Notebook and Dependencies

If you're seeing this in a running Jupyter notebook environment, you've likely already set up the virtual environment and installed the python dependencies, but if not, run the following:

```bash
$ python3 -m venv venv
$ source venv/bin/activate
$ pip install -r requirements.txt
$ jupyter notebook
```

### API Keys

You'll also need to get an API key from Maker Suite for the Gemini Ultra model. Go on over to [Google Maker Suite](makersuite.google.com) and get an API key for Gemini Ultra. Your API key will be attached to a GCP Project as well, so make note of both the API Key and the GCP Project ID associated with the key.

Copy `env.sample` into a hidden system file called `.env`. And replace the relevant environment variables with their respective values.

### Load the Environment Variables into the Jupyter Notebook

In [40]:
import os
from dotenv import load_dotenv
load_dotenv() 

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
GEMINI_ULTRA_API_KEY=os.getenv("GEMINI_ULTRA_API_KEY")
GCP_PROJECT_ID=os.getenv("GCP_PROJECT_ID")


### Create a Weaviate Client

In [41]:
import weaviate
import weaviate.classes as wvc
print(weaviate.__version__)

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
        "X-PaLM-Api-Key": GEMINI_ULTRA_API_KEY,
    }
)

4.4.4.dev3+g1b381573


### Delete an existing schema

Optionally, if you've run this notebook before, you can delete the existing schema to start fresh. 

In [42]:
client.collections.delete("Question")

### Define a Schema

With a fresh database, you can define the schema as shown below.

In [ ]:

questions = client.collections.create(
    name="Question",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
    # vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_palm(project_id=GCP_PROJECT_ID, model_id="textembedding-gecko-001", api_endpoint="generativelanguage.googleapis.com"),  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    generative_config=wvc.config.Configure.Generative.palm(project_id=GCP_PROJECT_ID, model_id="gemini-ultra", api_endpoint="generativelanguage.googleapis.com")  # Ensure the `generative-openai` module is used for generative queries
)


### Insert the data

We'll use a list of Jeopardy style questions from the [weaviate-tutorials/quicakstart](https://github.com/weaviate-tutorials/quickstart) repository. This JSON contains an array of objects with the following attributes: question, answer, and category. This array of objects will be batch loaded into your Weaviate instance.

In [44]:

import json
import requests
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

question_objs = list()
for i, d in enumerate(data):
    question_objs.append({
        "answer": d["Answer"],
        "question": d["Question"],
        "category": d["Category"],
    })

questions = client.collections.get("Question")
questions.data.insert_many(question_objs)  # This uses batching under the hood


BatchObjectReturn(all_responses=[UUID('7a8ef181-591d-4ac8-82ef-1ce361cde587'), UUID('ea66848a-1427-4867-9302-4a8e77f7328d'), UUID('f65802d6-aaee-40e5-b695-9559eff31bea'), UUID('43a52d9a-78e4-484c-9877-ae73628e4be0'), UUID('f6719845-2eb2-485b-90c2-3bc1a0177cdb'), UUID('8c1faa2f-b9fd-4f19-b606-f7e97e012d3d'), UUID('925f72d3-b86c-421f-98c2-3e2ada44cfab'), UUID('1958e0a2-e35e-4048-a906-d4af4496ed75'), UUID('479824de-4b69-4f62-b8f8-623b38ba5c9d'), UUID('71d8b6d6-fb2f-4d26-a3f6-a1c76c297452')], elapsed_seconds=0.7714881896972656, errors={}, uuids={0: UUID('7a8ef181-591d-4ac8-82ef-1ce361cde587'), 1: UUID('ea66848a-1427-4867-9302-4a8e77f7328d'), 2: UUID('f65802d6-aaee-40e5-b695-9559eff31bea'), 3: UUID('43a52d9a-78e4-484c-9877-ae73628e4be0'), 4: UUID('f6719845-2eb2-485b-90c2-3bc1a0177cdb'), 5: UUID('8c1faa2f-b9fd-4f19-b606-f7e97e012d3d'), 6: UUID('925f72d3-b86c-421f-98c2-3e2ada44cfab'), 7: UUID('1958e0a2-e35e-4048-a906-d4af4496ed75'), 8: UUID('479824de-4b69-4f62-b8f8-623b38ba5c9d'), 9: UUID('71

### Semantic Search

Before we get into doing generative query with Google Gemini Ultra, let's see how a simple semantic search query looks. We'll pass these results to Gemini Ultra in the Generative Search cell that comes after this one.

In [45]:
response = questions.query.near_text(
    query="biology",
    limit=2
)

print(response.objects[0].properties)  # Inspect the first object


{'answer': 'DNA', 'question': 'In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance', 'category': 'SCIENCE'}


#### Generative Search with Gemini Ultra

Now let's try a generative search against the results of a semantic search with Gemini Ultra as the language model.

In [46]:
questions = client.collections.get("Question")

response = questions.generate.near_text(
    query="biology",
    limit=2,
    single_prompt="Explain {answer} as you might to a five-year-old."
)

print(response.objects[0].generated)  # Inspect the generated text

Imagine**What is DNA?**

Think of your body as a big building made of tiny blocks called cells. And inside each cell is a special code that tells your body how to make you, YOU! That code is called DNA.

**What does DNA look like?**

It looks like a long, twisty ladder. The sides of the ladder are like the walls of a building, and the steps of the ladder are like the letters of the code.

**What do the letters mean?**

There are four letters in the DNA code: A, C, G, and T. These letters stand for different chemicals that make up your body. The order of these letters is like a secret message that tells your body how to build your hair, your eyes, your nose, and everything else that makes you special.

**How does DNA make me different from everyone else?**

Everyone has DNA, but the order of the letters in our DNA is different for each person. That's why you don't look exactly like your brother or sister, or anyone else in the world!

**Fun Fact:**

If you could stretch out all the DNA 

In [29]:
client.close()  # Close client gracefully